In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('..'))
from src.appr_core import GridEnvironment, enrich_data_with_forecast
from src.appr_agent import DQNAgent # PyTorch Agent


In [ ]:

# Re-importar dependencias de TensorFlow/Keras

# --- RECARGA DE PARÁMETROS DEL ENTORNO (DEBE COINCIDIR CON EL NOTEBOOK ANTERIOR) ---
# Si el notebook se ejecuta independientemente, necesitamos redefinir el DataFrame y parámetros:
try:
    # Intenta usar el df creado en el notebook anterior
    df 
except NameError:
    print("DataFrame 'df' no encontrado. Re-generando datos del caso de estrés...")
    # Si falla, generamos los datos de nuevo (simplificado para la prueba)
    HORIZONTE_DIAS = 3
    CAPACIDAD_INSTALADA_SOLAR = 100.0
    CAPACIDAD_RESTRINGIDA = 80.0
    HORAS = 24 * HORIZONTE_DIAS
    index = pd.date_range(start='2024-12-21', periods=HORAS, freq='H')
    df = pd.DataFrame(index=index)
    # Simulación básica para que el código corra
    df['Demanda_MW'
] = 70 + 10 * np.sin(2 * np.pi * (df.index.hour) / 24)
    df['Generacion_Solar_MW'
] = 90 + 30 * np.sin(2 * np.pi * (df.index.hour - 6) / 48) 
    df['Exceso_Solar'
] = df['Generacion_Solar_MW'
] - CAPACIDAD_RESTRINGIDA
    df['Curtailment_Baseline_MW'
] = df['Exceso_Solar'
].apply(lambda x: max(0, x))

# --- PARÁMETROS DE LA BATERÍA ---
CAPACIDAD_BATERIA_MWh = 60.0
TASA_MAX_MW = 20.0 # Tasa máxima de carga/descarga permitida por hora
SOC_INICIAL = CAPACIDAD_BATERIA_MWh / 2  # Empezamos a medio cargar

# ====================================================================
# PARTE 1: DEFINICIÓN DEL ENTORNO RL (GridEnvironment)
# ====================================================================

# GridEnvironment imported from src.appr_core
# --- ENRICH DATA ---
df = enrich_data_with_forecast(df)

env = GridEnvironment(
    df=df, 
    capacity_limit=CAPACIDAD_RESTRINGIDA, 
    battery_capacity=CAPACIDAD_BATERIA_MWh,
    battery_rate=TASA_MAX_MW,
    training_mode=True
)

print(f"Entorno RL inicializado. Estado inicial (SOC): {env.soc:.2f} MWh.")


# ====================================================================
# PARTE 2: IMPLEMENTACIÓN DEL AGENTE DQN (TensorFlow/Keras)
# ====================================================================

# --- Definición de Modelos y Hiperparámetros ---
state_size = env.state_space_size
action_size = env.action_space_size

# Hiperparámetros
EPISODES = 150 # Reducido para una prueba rápida de concepto
GAMMA = 0.95
EPSILON_START = 1.0
EPSILON_END = 0.05
EPSILON_DECAY = (EPSILON_START - EPSILON_END) / EPISODES

# Memoria de Experiencia
memory = deque(maxlen=20000)

# Función para construir la Q-Network
def build_dqn(input_shape, output_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='linear')
])
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))
    return model

# Inicialización de Redes
q_network = build_dqn(state_size, action_size)
target_network = build_dqn(state_size, action_size)
target_network.set_weights(q_network.get_weights()) # Sincronizar pesos iniciales

# --- Bucle de Entrenamiento ---
history = []
epsilon = EPSILON_START
print(f"\nIniciando entrenamiento del APPR para {EPISODES} episodios...")

for e in range(EPISODES):
    state = env.reset()
    episode_reward = 0
    done = False
    
    while not done:
        # 1. Elección de Acción (Exploración vs. Explotación)
        if np.random.rand() <= epsilon:
            action = random.randrange(env.action_space_size) # Exploración
        else:
            # Optimización: Usar __call__ en lugar de predict para inferencia rápida
    # Usar agente para predicción
    action = agent.act(state, training=False)
    0
]) # Explotación

        # 2. Ejecutar la acción
        next_state, reward, done, _ = env.step(action)
        
        # 3. Almacenar experiencia
        memory.append((state, action, reward, next_state, done))
        
        state = next_state
        episode_reward += reward
        
        # 4. Entrenamiento (Replay Batch)
        if len(memory) > 100:
            minibatch = random.sample(memory, min(64, len(memory)))
            
            states, actions, rewards, next_states, dones = zip(*minibatch)
            states = np.array(states)
            next_states = np.array(next_states)
            
            # Cálculo del Target Q-Value
            target_q_next = target_network(next_states, training=False).numpy().max(axis=1)
            target_q = rewards + GAMMA * target_q_next * (1 - np.array(dones))
            
            # Actualización de la Q-Network
            target_f = q_network(states, training=False).numpy()
            for i in range(len(actions)):
                target_f[i
][actions[i
    ]
] = target_q[i
]
            
            # Optimización: train_on_batch es mucho más rápido que fit para un solo lote
            q_network.train_on_batch(states, target_f)
        
        if done:
            break

    # 5. Actualización de Parámetros
    epsilon = max(EPSILON_END, epsilon - EPSILON_DECAY)
    
    if e % 25 == 0:
        target_network.set_weights(q_network.get_weights())
        
    history.append(episode_reward)
    
    if e % 25 == 0 or e == EPISODES - 1:
        print(f"Episodio {e}/{EPISODES} | Recompensa Total: {episode_reward:.2f} | Epsilon: {epsilon:.3f}")


# ====================================================================
# PARTE 3: EVALUACIÓN FINAL DEL APRENDIZAJE
# ====================================================================

# 1. Ejecutar el entorno con la política entrenada (Explotación total)
env.training_mode = False # Habilitar logging para evaluación
env.reset()
state = env._get_state()
done = False

while not done:
    # Usar la política entrenada (Explotación pura)
    # Usar agente para predicción
    action = agent.act(state, training=False)
    0
])
    
    # Ejecutar la acción
    state, _, done, _ = env.step(action)

# 2. Calcular el Desperdicio total del APPR
total_curtailment_appr = df['Curtailment_APPR'
].sum()
total_curtailment_baseline = df['Curtailment_Baseline_MW'
].sum()

print("\n" + "="*50)
print("           RESULTADOS FINALES DE LA OPTIMIZACIÓN          ")
print("="*50)
print(f"1. BaseLine (Corte Inmediato): {total_curtailment_baseline:.2f} MWh Desperdiciados")
print(f"2. APPR (DQN Optimizado):     {total_curtailment_appr:.2f} MWh Desperdiciados")

if total_curtailment_baseline > 0:
    reduccion = ((total_curtailment_baseline - total_curtailment_appr) / total_curtailment_baseline) * 100
    print(f"\n✅ Mitigación lograda: {reduccion:.2f}% de reducción del desperdicio.")
else:
    print("\nAdvertencia: El escenario de estrés no fue lo suficientemente severo para el baseline.")


# 3. Visualización de la Recompensa y el SOC
fig, (ax1, ax2) = plt.subplots(2,
1, figsize=(15,
8), sharex=True)

# Gráfico 1: Recompensa durante el entrenamiento
ax1.plot(history, label='Recompensa Total por Episodio', color='blue')
ax1.set_title('Progreso del Entrenamiento del APPR (Recompensa)')
ax1.set_ylabel('Recompensa Acumulada (Mayor es Mejor)')
ax1.grid(True, alpha=0.4)

# Gráfico 2: Comparación de Inyección (Solo 24h críticas para claridad)
df_plot = df.iloc[
    24: 48
].copy() # Tomamos el segundo día como ejemplo
df_plot['Inyeccion_APPR'
] = df_plot['Generacion_Solar_MW'
] - df_plot['Curtailment_APPR'
]
df_plot['Inyeccion_Baseline_MW'
] = df_plot['Generacion_Solar_MW'
] - df_plot['Curtailment_Baseline_MW'
]


ax2.plot(df_plot.index, df_plot['Demanda_MW'
], label='Demanda', color='gray', alpha=0.7)
ax2.plot(df_plot.index, df_plot['Inyeccion_Baseline_MW'
], label='Inyección Baseline', color='green', linestyle=':')
ax2.plot(df_plot.index, df_plot['Inyeccion_APPR'
], label='Inyección APPR Optimizada', color='purple', linewidth=2)
ax2.axhline(CAPACIDAD_RESTRINGIDA, color='red', linestyle='--', label='Límite de Transmisión')
ax2.set_title('Comparación de Despacho (Ejemplo de 1 Día Crítico)')
ax2.set_ylabel('Potencia (MW)')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.4)

# Eje secundario para ver el estado de carga (SOC)
ax3 = ax2.twinx()
ax3.plot(df_plot.index, df_plot['SOC_t'], label='SOC Batería', color='orange', linestyle='--', alpha=0.5)
ax3.set_ylabel('Energía (MWh)')
ax3.legend(loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
# --- INICIALIZACIÓN DEL AGENTE PYTORCH ---
state_size = env.state_space_size
action_size = env.action_space_size

agent = DQNAgent(state_size, action_size)
print('Agente DQN (PyTorch) inicializado.')
